In [4]:
import pandas as pd
import itertools
import re
import tkinter as tk
from tkinter import messagebox
from tkinter.scrolledtext import ScrolledText

class Course:
    def __init__(self, course_name, course_id, units, times, category, level):
        self.course_name = course_name
        self.course_id = course_id
        self.units = units
        self.times = times
        self.category = category
        self.level = level

def split_time_blocks(start_time, end_time):
    time_blocks = []
    for slot_start, slot_end in time_slots:
        if start_time < slot_end and end_time > slot_start:
            time_blocks.append((slot_start, slot_end))
    return time_blocks

def adjust_time_blocks_for_search(df):
    df['Processed Times'] = df.apply(lambda row: [f"{row['Days']} {row['Start time']}-{row['End time']}"], axis=1)
    return df

time_slots = [
    (8.3, 9.45), (10.0, 11.25), (11.75, 13.0),
    (13.25, 14.3), (14.45, 16.0), (16.15, 17.3),
    (18.0, 19.25), (19.5, 20.75)
]

avoid_time_ranges = {
    1: (8.30, 9.45),
    2: (10.00, 12.00),
    3: (13.15, 14.30),
    4: (14.45, 17.00),
    5: None 
}

def process_course_times(df):
    df['Time Blocks'] = df.apply(lambda row: split_time_blocks(row['Start time'], row['End time']), axis=1)
    df = adjust_time_blocks_for_search(df)
    return df

def extract_category_and_level(title):
    category_match = re.match(r'[A-Za-z]+', title)
    category = category_match.group(0) if category_match else ""
    
    level_match = re.search(r'\d+', title)
    level = str(int(level_match.group(0)) // 100 * 100) if level_match else '0'
    
    return category, level

def create_course_db(df):
    course_db = []
    for _, row in df.iterrows():
        category, level = extract_category_and_level(row['Title'])
        course_db.append(Course(
            row['Title'],
            row['Section'],
            row['Unit'],
            row['Processed Times'],
            category,
            level
        ))
    return course_db

def combine_courses(df):
    combined_courses = []
    columns = df.columns
    current_main_course = None
    current_combinations = []

    for _, row in df.iterrows():
        section = row['Section']
        
        if "LEC" in section or "SEM" in section:
            if current_main_course is not None:
                combined_courses.extend(current_combinations)
            current_main_course = row.copy()
            current_combinations = [[current_main_course]]
        elif "LAB" in section or "REC" in section:
            if current_main_course is not None:
                new_combinations = []
                for combination in current_combinations:
                    new_combination = combination.copy()
                    new_combination.append(row.copy())
                    new_combinations.append(new_combination)
                current_combinations = new_combinations
    
    if current_combinations:
        combined_courses.extend(current_combinations)

    combined_courses_flat = []
    for combination in combined_courses:
        combined_row = {}
        for col in columns:
            combined_row[col] = combination[0][col]
        for additional_course in combination[1:]:
            combined_row['Unit'] += additional_course['Unit']
            combined_row['Processed Times'] += additional_course['Processed Times']
        combined_courses_flat.append(combined_row)

    combined_courses_df = pd.DataFrame(combined_courses_flat)
    return combined_courses_df

def has_time_conflict(course1, course2):
    for time_slot1 in course1.times:
        for time_slot2 in course2.times:
            if time_slot1 == time_slot2:
                return True
    return False

def time_avoided(course, avoid_time):
    for time in course.times:
        start_time = float(re.search(r'(\d+\.\d+)', time).group(1))
        if avoid_time and avoid_time[0] <= start_time < avoid_time[1]:
            return True
    return False

def search_course_combinations(mandatory_courses, category_requirements, course_db, avoid_time):
    selected_courses = [course for course in course_db if course.course_name in mandatory_courses]
    grouped_courses = {category: [] for category in category_requirements}

    for course in course_db:
        if course.category in category_requirements and course.level == category_requirements[course.category]:
            grouped_courses[course.category].append(course)

    all_combinations = list(itertools.product(*grouped_courses.values())) if grouped_courses else [[]]

    valid_combinations = []
    for combination in all_combinations:
        full_combination = selected_courses + list(combination)
        if not any(has_time_conflict(course1, course2) for course1, course2 in itertools.combinations(full_combination, 2)):
            if avoid_time:
                if not any(time_avoided(course, avoid_time) for course in full_combination):
                    valid_combinations.append(full_combination)
            else:
                valid_combinations.append(full_combination)

    return valid_combinations

def print_combinations(combinations):
    if not combinations:
        return "No valid course combinations found.\n"
    result = ""
    for i, combination in enumerate(combinations, 1):
        result += f"Plan {i}:\n"
        for course in combination:
            result += f"{course.course_name} ({course.units} units)\nTime slots:\n"
            for time in course.times:
                result += f"  {time}\n"
    return result

def create_ui():
    window = tk.Tk()
    window.title("Course Selection System")
    window.geometry("600x600")

    label1 = tk.Label(window, text="Enter mandatory course IDs (comma separated):")
    label1.pack()
    mandatory_entry = tk.Entry(window)
    mandatory_entry.pack()

    label2 = tk.Label(window, text="Enter course categories and levels (format: ECON,200 or done to finish, one per line):")
    label2.pack()
    category_text = tk.Text(window, height=5)
    category_text.pack()

    label3 = tk.Label(window, text="Choose time to avoid:\n1. 8:30-9:45\n2. 10:00-12:00\n3. 1:15-2:30\n4. 2:45-5:00\n5. No time constraints")
    label3.pack()
    avoid_time_entry = tk.Entry(window)
    avoid_time_entry.pack()

    result_box = ScrolledText(window, height=20, width=80)
    result_box.pack()

    def on_search():
        mandatory_courses = mandatory_entry.get().split(',')
        category_lines = category_text.get("1.0", tk.END).strip().splitlines()
        category_requirements = {line.split(',')[0].strip(): line.split(',')[1].strip() for line in category_lines if line}
        avoid_choice = int(avoid_time_entry.get())
        avoid_time = avoid_time_ranges.get(avoid_choice, None)

        file_path = "final_courses.csv"  # Change to your file path
        courses_df = pd.read_csv(file_path)
        processed_df = process_course_times(courses_df)
        combined_courses_df = combine_courses(processed_df)
        course_db = create_course_db(combined_courses_df)

        combinations = search_course_combinations(mandatory_courses, category_requirements, course_db, avoid_time)
        result_text = print_combinations(combinations)

        result_box.delete(1.0, tk.END)
        result_box.insert(tk.END, result_text)

    search_button = tk.Button(window, text="Search Course Combinations", command=on_search)
    search_button.pack()

    window.mainloop()

if __name__ == "__main__":
    create_ui()
